In [ ]:
# -*- coding: utf-8 -*-
"""
import pandas as pd
import numpy as np
import datetime
import time

df = pd.read_csv(r"\CSV\RoutineEcho_CompleteSet.csv",) #nrows=50000)

########################
# Cleaning
########################

df['StudyDate'] = pd.to_datetime(df.StudyDate, format='%Y-%m-%d')

# Filtering out studies by date
bottom_date = datetime.datetime(2016, 6, 1)
cond_1 = df.StudyDate >= bottom_date

# Filtering by status
cond_2 = (df.StudyStatus == 'Final') | (df.StudyStatus == 'Revised')

# Building clean dataset
df = df[cond_1 & cond_2]

##############################
# Checking for missing order IDs
##############################
null_IDs = df.OrderID.isnull()
random_vector = np.arange(0,sum(null_IDs),1,dtype=int)
list_missing_ord_id = [str(x) + 'MISS' for x in random_vector]

df.loc[null_IDs, 'OrderID'] = list_missing_ord_id
#[1]


#We are trying to map a multi-dimensional array into one cell location

#print(len(df.OrderID))
#print(len(list_missing_ord_id))


##############################
# Mod. Duplicates Order ID
##############################

flag_duplicates = df.OrderID.duplicated()
curr_order_id = list(df.OrderID[flag_duplicates])
new_order_id = [str(x) + "_DUP" for x in curr_order_id]
df.loc[flag_duplicates, 'OrderID'] = new_order_id

##############################
# Data restructuring
##############################
s_keys = df.OrderID

dict_long = {}
dict_long['PatientID'] = []
dict_long['PatientID2'] = []
dict_long['StudyDate'] = []
dict_long['OrderID'] = []
dict_long['Findings_Name'] = []
dict_long['Findings_Value'] = []

key_columns = ['PatientID', 'PatientID2', 'StudyDate', 'OrderID']

start_time = time.time()

other_columns = []
for c_col in list(df.columns):
    if c_col not in key_columns:
        if c_col[:8] == 'FINDINGS':
            other_columns.append(c_col)

for i, key in enumerate(s_keys):
    
    df_loc = df[df.OrderID == key]
    
    PAT_ID = df_loc.PatientID.to_numpy()[0]
    PAT_ID2 = df_loc.PatientID2.to_numpy()[0]
    ST_DATE = df_loc.StudyDate.to_numpy()[0]
    ORD_ID = df_loc.OrderID.to_numpy()[0]
    
    for c_col in other_columns:
        if ~df_loc[c_col].isnull().iloc[0]:
            dict_long['PatientID'].append(PAT_ID)
            dict_long['PatientID2'].append(PAT_ID2)
            dict_long['StudyDate'].append(ST_DATE)
            dict_long['OrderID'].append(ORD_ID)
            dict_long['Findings_Name'].append(c_col)
            dict_long['Findings_Value'].append(df_loc[c_col].to_numpy()[0])
            
    if i%100 == 0: 
        print('Done with {0} order IDs'.format(i))
        elapsed_time = time.time() - start_time
        number_files_missing = len(s_keys) - i
        missing_time = elapsed_time*number_files_missing/(i+1)
        print('Missing time: {0:.1f} minutes'.format(missing_time/60))
        

# Converting output into a dataframe
df_out = pd.DataFrame(dict_long)

df_out.to_csv(r'CSV\RoutineEcho_ReshapedLong.csv')

##############################
# Finding columns
##############################
base_string = "mitral regurgitation"
charac_org = ["No evidence of", "Mild", "Mild to moderate", "Moderate", "Moderate to severe", "Severe", "Unable to assess severity of"]
char_rec = ["A trace of", "Clinically insignificant"]
charac = charac_org + char_rec

dict_var = {}
dict_var['OrderID'] = []
dict_var['MR_Severity'] = []

for c in charac:
    
    print(c)
    
    search_string = c + " " + base_string
    flags = df_out['Findings_Value'].str.contains(search_string, case=True)
        
    print(df_out[flags].Findings_Value.iloc[0:10])
    ord_id = list(df_out[flags].OrderID)
    if c in char_rec:
        severity = ["Mild"] * len(ord_id)
    else:
        severity = [c] * len(ord_id)
    for i, _ in enumerate(ord_id):
        dict_var['OrderID'].append(ord_id[i])
        dict_var['MR_Severity'].append(severity[i])

# Creating variable dictionary
df_var = pd.DataFrame(dict_var)

# Created a new variable
## Code for the new variable ###
# Let's say the new variable is in df_var_2

#df_var = df_var.merge(df_var_2, on='OrderID', how='outer')

# Create the output dataframe
df_export = df[key_columns].merge(df_var, on='OrderID', how='left')
df_export.to_csv(r"S:\Data Team\NASIR\Routine Echo\Data\CSV\Test\test_out.csv")

## Checks
# unique_order_IDs = len(df_out[flags].OrderID.unique())
# unique_entries = sum(flags)
# unique_patients = len(df_out[flags].PatientID.unique())
# lu = df_out[flags].Findings_Name.unique()
# sum(df_out[flags].Findings_Name == lu[0])
# sum(df_out[flags].Findings_Name == lu[1])
# sum(df_out[flags].Findings_Name == lu[2])

 #usecols=list(np.arange(0,10,1,dtype=int)))

#df = dfc

#print(dfc.head(3))
#print(dfc.tail(3))

#dfc.columns    //Read each column header
#print(dfc.columns)
#dfc['OrderID'] ...//Read each column
#print(dfc['OrderID'])
